In [81]:
import shap
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import math
# import eli5

In [82]:
#pip install "shap"

In [83]:
# Read data
IndicesPd = pd.read_excel("File_5_-_IoD2019_Scores.xlsx", sheet_name="IoD2019 Scores", header= 0)

In [84]:
IndicesPd.columns

Index(['LSOA code (2011)', 'LSOA name (2011)',
       'Local Authority District code (2019)',
       'Local Authority District name (2019)', 'Unnamed: 4', 'Unnamed: 5',
       'Index of Multiple Deprivation (IMD) Score', 'Unnamed: 7',
       'Income Score (rate)', 'Employment Score (rate)',
       'Education, Skills and Training Score',
       'Health Deprivation and Disability Score', 'Crime Score',
       'Barriers to Housing and Services Score', 'Living Environment Score',
       'Income Deprivation Affecting Children Index (IDACI) Score (rate)',
       'Income Deprivation Affecting Older People (IDAOPI) Score (rate)',
       'Children and Young People Sub-domain Score',
       'Adult Skills Sub-domain Score',
       'Geographical Barriers Sub-domain Score',
       'Wider Barriers Sub-domain Score', 'Indoors Sub-domain Score',
       'Outdoors Sub-domain Score', 'Income Rank (where 1 is most deprived)',
       '32845-trueRank'],
      dtype='object')

In [85]:
y = IndicesPd["Index of Multiple Deprivation (IMD) Score"]
income_score = IndicesPd['Income Score (rate)']
employment_score = IndicesPd['Employment Score (rate)']
edu_skill_train_score = IndicesPd['Education, Skills and Training Score']
health_scores = IndicesPd['Health Deprivation and Disability Score']
crime_scores = IndicesPd['Crime Score']
house_service_scores = IndicesPd['Barriers to Housing and Services Score']
live_environ_scores=IndicesPd['Living Environment Score']
X = pd.concat([income_score, employment_score,edu_skill_train_score,health_scores, crime_scores, house_service_scores, live_environ_scores] , axis=1)

In [86]:
#trueRank=IndicesPd['32845-trueRank']

In [87]:
ranks=X.rank()

In [88]:
ranks.iloc[:,1].sort_values().head(50)

32407     1.0
32731     4.5
7659      4.5
4568      4.5
12974     4.5
2838      4.5
17463     4.5
32729     9.0
13949     9.0
2757      9.0
31825    15.5
31822    15.5
2800     15.5
2773     15.5
24442    15.5
29888    15.5
29685    15.5
27857    15.5
14269    15.5
7656     15.5
32024    27.5
2754     27.5
2772     27.5
32071    27.5
32642    27.5
9380     27.5
32649    27.5
32706    27.5
28820    27.5
4571     27.5
29889    27.5
4570     27.5
32752    27.5
32748    27.5
4569     41.0
32170    41.0
27838    41.0
27795    41.0
32758    41.0
2795     41.0
8158     41.0
4606     41.0
4595     41.0
4995     41.0
4572     41.0
7661     41.0
4567     41.0
32756    53.5
32477    53.5
2835     53.5
Name: Employment Score (rate), dtype: float64

In [89]:
x=-23*np.log((1-ranks/32844*(1-math.exp(-100/23))))
IMD = np.dot(np.array([0.225,0.225,0.135,0.135,0.093,0.093,0.093]),np.array(x).T)
# IMD = pd.DataFrame({"IMD":np.dot(np.array([0.225,0.225,0.135,0.135,0.093,0.093,0.093]),np.array(x).T)})    
diff=y-IMD
diff.sort_values(ascending=True)

4966    -0.081472
11328   -0.080692
9098    -0.072974
12416   -0.072907
9039    -0.072465
           ...   
1800     0.075258
6560     0.077372
30302    0.080056
24239    0.081062
2790     0.088025
Name: Index of Multiple Deprivation (IMD) Score, Length: 32844, dtype: float64

In [90]:
diff.describe()

count    32844.000000
mean         0.000054
std          0.023079
min         -0.081472
25%         -0.015913
50%          0.000179
75%          0.015930
max          0.088025
Name: Index of Multiple Deprivation (IMD) Score, dtype: float64

In [91]:
IMD = pd.DataFrame({"calculated_IMD_score":IMD})
IMD

,calculated_IMD_score
0,6.206461
1,5.113783
2,19.413750
3,28.623760
4,19.795692
...,...
32839,74.477868
32840,59.615837
32841,21.882057
32842,66.725581


In [92]:
calculated_IMD_ranking=IMD.rank()
calculated_IMD_ranking.columns=["calculated_IMD_ranking"]

In [93]:
IMDs= pd.concat([IMD,calculated_IMD_ranking],axis=1)
IMDs

,calculated_IMD_score,calculated_IMD_ranking
0,6.206461,3640.0
1,5.113783,2444.0
2,19.413750,17945.0
3,28.623760,24157.0
4,19.795692,18309.0
...,...,...
32839,74.477868,32730.0
32840,59.615837,31901.0
32841,21.882057,19992.0
32842,66.725581,32425.0


In [94]:
IMDs= IMD.join(calculated_IMD_ranking)
IMDs

,calculated_IMD_score,calculated_IMD_ranking
0,6.206461,3640.0
1,5.113783,2444.0
2,19.413750,17945.0
3,28.623760,24157.0
4,19.795692,18309.0
...,...,...
32839,74.477868,32730.0
32840,59.615837,31901.0
32841,21.882057,19992.0
32842,66.725581,32425.0


In [104]:
IMDs[IMDs.calculated_IMD_ranking==1000]

,calculated_IMD_score,calculated_IMD_ranking
12087,3.582422,1000.0


In [44]:
IMDs.to_csv("IMDs.csv")

It's the end of the calculation for IMD.
The next part is the TreeSHAP wrote before.

In [8]:
mod = sk.ensemble.RandomForestRegressor(n_estimators = 100, random_state = 42)
mod.fit(X,y)

# explainer = shap.TreeExplainer(mod, X, model_output = "margin", feature_pertubation = "interventional")
# shap_values = explainer.shap_values(X)

RandomForestRegressor(random_state=42)

In [10]:
# explainer = shap.TreeExplainer(mod, X, model_output = "margin", feature_pertubation = "interventional")
explainer = shap.TreeExplainer(mod)

In [ ]:
shap_values = explainer.shap_values(X)

In [1]:
# visualize the first prediction's explanation 
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

NameError: name 'shap' is not defined

In [8]:

i = 18

# Explain individual predictions
shap.force_plot(explainer.expected_value[1], shap_values[1][i,:], X.iloc[i,:], matplotlib = True)

NameError: name 'explainer' is not defined

In [ ]:
# visualize the training set predictions
shap.force_plot(explainer.expected_value, shap_values, X)

In [ ]:
shap_values = shap.TreeExplainer(model).shap_values(X)
shap.summary_plot(shap_values, X, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, X)

In [ ]:
shap.dependence_plot('Income Score (rate)', shap_values, X_train)